In [ ]:
TANGO_HOST = "10.164.10.219:10000"
RECEPTORS = ["SKA001", "SKA036"]

In [ ]:
import json
import os
from datetime import datetime
from time import sleep

from tango import AttributeProxy, DeviceProxy

os.environ["TANGO_HOST"] = TANGO_HOST

csp_controller = "mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
sdp_subarray = "mid-sdp/subarray/01"
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"

csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)
sdp_subarray_dp = DeviceProxy(sdp_subarray)
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

SIMULATED_DELAY = False


In [ ]:
sleep(3)

csp_controller_dp.adminMode = 0

sleep(6)

with open(
    "../../data/mid_telescope/cbf/sys_params/initial_system_param.json", "r", encoding="utf-8"
) as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))

sleep(3)
print(csp_controller_dp.sourceDishVccConfig)
sleep(3)
csp_controller_dp.commandTimeout = 100

csp_controller_dp.cbfSimulationMode = False
sleep(3)
csp_master_leaf_node_dp.On()

# WAIT 100 Seconds

### CSP Assign Resources

In [ ]:
with open(
    "../../data/mid_telescope/csp/assign_resources.json", "r", encoding="utf-8"
) as json_data:
    assign_resources_csp = json.load(json_data)
    assign_resources_csp["dish"]["receptor_ids"] = RECEPTORS
    print(assign_resources_csp)
    csp_subarray_leaf_node_dp.AssignResources(json.dumps(assign_resources_csp))

sleep(5)

print(csp_dp.obsState)

### SDP Assign Resources

In [ ]:
sdp_subarray_leaf_node_dp.On()

sleep(3)

with open(
    "../../data/mid_telescope/tmc/assign_resources.json", "r", encoding="utf-8"
) as json_data:
    d = json.load(json_data)
    sdp_assign_resources = d["sdp"]
    sdp_assign_resources["resources"]["receptors"] = RECEPTORS
    sdp_subarray_leaf_node_dp.AssignResources(json.dumps(sdp_assign_resources))

sleep(10)
print(sdp_subarray_dp.obsState)

In [ ]:
receive_addresses = json.loads(sdp_subarray_dp.receiveAddresses)
print(receive_addresses)
print(receive_addresses["science"]["vis0"]["host"][0][1])

### CSP Configure Scan

In [ ]:
receive_addresses = json.loads(sdp_subarray_dp.receiveAddresses)

sleep(1)

with open("../../data/mid_telescope/tmc/configure_scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_configure_scan = d["csp"]
    csp_configure_scan["cbf"]["fsp"][0]["output_host"] = [[]]
    if SIMULATED_DELAY == True:
        csp_configure_scan["cbf"]["delay_model_subscription_point"] = "ska_mid/tm_leaf_node/csp_subarray_01/delayModel"
    else:
        csp_configure_scan["cbf"]["delay_model_subscription_point"] = "ska_mid/tm_leaf_node/csp_subarray01/delayModel"
    csp_configure_scan["cbf"]["fsp"][0]["output_host"][0] = [0, receive_addresses["science"]["vis0"]["host"][0][1]]
    csp_configure_scan["cbf"]["fsp"][0]["output_port"] = [[0, 21000, 1]]
    print(csp_configure_scan)
    csp_subarray_leaf_node_dp.Configure(json.dumps(csp_configure_scan))

sleep(12)
print(csp_dp.obsState)

### SIMULATE DELAY MODELS (OPTIONAL)

In [ ]:
from astropy.time import Time
from datetime import datetime, timedelta, timezone

SKA_EPOCH = "1999-12-31T23:59:28Z"
ska_epoch_utc = Time(SKA_EPOCH, scale="utc")
ska_epoch_tai = ska_epoch_utc.unix_tai

start_utc_time = Time(
        datetime.now(timezone.utc) + timedelta(seconds=5), scale="utc"
    )

target_epoch = start_utc_time.unix_tai - ska_epoch_tai

with open("../../data/mid_telescope/tmc/delay_model.json", encoding="utf-8") as f:
    delay_model = json.load(f)
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model))


tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

### SDP Configure Scan

In [ ]:
with open("../../data/mid_telescope/tmc/configure_scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_configure_Scan = d["sdp"]
    print(sdp_configure_Scan)
    sdp_subarray_leaf_node_dp.configure(json.dumps(sdp_configure_Scan))

sleep(3)
print(sdp_subarray_dp.obsState)

### SDP Scan

In [ ]:
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_dp.obsState)

### CSP Scan

In [ ]:
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_dp.obsState)

### Cleanup CSP

In [ ]:
csp_subarray_leaf_node_dp.EndScan()
sleep(4)
csp_subarray_leaf_node_dp.End()
sleep(4)
csp_subarray_leaf_node_dp.ReleaseAllResources()

### Cleanup SDP

In [ ]:
sdp_subarray_leaf_node_dp.EndScan()
#sdp_subarray_leaf_node_dp.End()
#sdp_subarray_leaf_node_dp.ReleaseAllResources()

### Turn Off CSP

In [ ]:
csp_master_leaf_node_dp.Off()